## Scrapy - imdb film verisi


Bu sayfadaki kodları hazırlarken takip eden kaynaktan faydalandım: https://app.datacamp.com/learn/courses/web-scraping-with-python

In [ ]:
#pip install scrapy

In [ ]:
# Import a scrapy Selector
from scrapy import Selector

# Import requests
import requests

url = 'https://m.imdb.com/chart/top'

# Create the string html containing the HTML source
html = requests.get(url).content

# Create the Selector object sel from html
sel = Selector(text = html)

# Print out the number of elements in the HTML document
print( "You have found: ", len(sel.xpath('//*')))

In [ ]:
type(html)

In [ ]:
type(sel)

In [ ]:
xpath_for_movienames ='//h4'

In [ ]:
sel.xpath(xpath_for_movienames).extract()

In [ ]:
len(sel.xpath(xpath_for_movienames).extract())

In [ ]:
xpath_for_movienames ='//h4/text()'

In [ ]:
movies = sel.xpath(xpath_for_movienames).extract()

In [ ]:
movies

#### "Strip" metoduyla elimizdeki metni temizleyelim

In [ ]:
movie_list=[]
for string in movies:
    cleaned_string = string.strip()
    if cleaned_string != '':
        movie_list.append(cleaned_string)
    
print(movie_list)

#### Kısa bir ara: strip fonksiyonu nasıl çalışır?

In [ ]:
string = '  xoxo love xoxo   '

# Leading and trailing whitespaces are removed
print(string.strip())

# All <whitespace>,x,o,e characters in the left
# and right of string are removed
print(string.strip(' xoe'))

# Argument doesn't contain space
# No characters are removed.
print(string.strip('stx'))

string = 'android is awesome'
print(string.strip('an'))

In [ ]:
len(movie_list)

#### Filmlerin sıralamasını ve yıllarını çekelim

In [ ]:
xpath_for_movieorder ='//h4/span[1]/text()'
movie_order = sel.xpath(xpath_for_movieorder).extract()

In [ ]:
xpath_for_moviedates ='//h4/span[2]/text()'
movie_date = sel.xpath(xpath_for_moviedates).extract()

##### İlk film için kopyaladığımız Xpath'ler:

//*[@id="chart-content"]/div[1]/div[1]/div/a

//*[@id="chart-content"]/div[1]/div[2]/div/a

//*[@id="chart-content"]/div[2]/div[1]/div/a

//*[@id="chart-content"]/div[2]/div[2]/div/a

…


#### Filmlerin linkleri

In [ ]:
xpath_for_movielink = '//*[@id="chart-content"]/div/div/div/a/@href'
movie_link = sel.xpath(xpath_for_movielink).extract()

In [ ]:
len(movie_link)

In [ ]:
movie_link

İndirdiğimiz link parçaları birbirinden ayrı, nasıl birleştirebiliriz?

In [ ]:
first_part_url = 'https://m.imdb.com'
last_part_url = '?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=df09bbba-7a44-41c0-bc85-426ba05a5574&pf_rd_r=R9N3Q0473JZET8YH4S2A&pf_rd_s=top-1&pf_rd_t=15506&pf_rd_i=top&ref_=m_chttp_tt_1'

In [ ]:
movie_link_merged = []

for i in movie_link:
    link = first_part_url + i + last_part_url
    movie_link_merged.append(link)

In [ ]:
len(movie_link_merged)

#### İndirdiklerimizi saklayalım

In [ ]:
dictimdb = {'movie_order':movie_order, 'movie_date':movie_date, 'movie_list':movie_list, 'movie_link_merged': movie_link_merged}

In [ ]:
data_imdb = pd.DataFrame(dictimdb)

In [ ]:
data_imdb['movie_link_merged'][0]

#### Biraz daha ileri gidebilir miyiz?

Filmlerin oyuncularını da indirelim.

In [ ]:
url = data_imdb['movie_link_merged'][0]

In [ ]:
html = requests.get(url).content

sel = Selector(text = html)

print( "You have found: ", len(sel.xpath('//*')))

##### İlk dört oyuncu için kopyaladığımız Xpath'ler:

//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[1]/div[2]/a

//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[2]/div[2]/a

//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[3]/div[2]/a

//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[4]/div[2]/a


In [ ]:
xpath_for_movieplayers ='//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div/div[2]/a/text()'
movie_players_first_movie = sel.xpath(xpath_for_movieplayers).extract()

In [ ]:
movie_players_first_movie

In [ ]:
players_list=[]

In [ ]:
for i in range(5):
    url = data_imdb['movie_link_merged'][i]
    html = requests.get(url).content
    sel = Selector(text = html)
    movie_players = sel.xpath(xpath_for_movieplayers).extract()
    players_list.append(movie_players)

In [ ]:
players_list[4]